<a href="https://colab.research.google.com/github/Berenice2018/DeepLearning/blob/master/section_2__Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning
copied from Kaggle. see https://www.kaggle.com/berenice18/pysyft-section-2-federated-learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [0]:
import torch as th
import syft as sy

In [0]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [0]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [0]:
hook = sy.TorchHook(th)

In [0]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [0]:
bob._objects

{18138202361: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [0]:
bob._objects

{25026462847: tensor([1, 2, 3, 4, 5])}

''' performing on x: 
it is going to send a message to self.location, 
the worker finds the tensor with the ID x.id_at_location
execute the command'''

In [0]:
x.location

<VirtualWorker id:bob #objects:1>

In [0]:
x.id_at_location

25026462847

In [0]:
x.id

36028299757

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
x

(Wrapper)>[PointerTensor | me:36028299757 -> bob:25026462847]

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects



{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# try this project here!
bob = sy.VirtualWorker(hook, 'bob')
ada = sy.VirtualWorker(hook, 'ada')

mytensor = th.Tensor([2,3,5,7])

In [0]:
tensor_pointer = mytensor.send(bob, ada)

In [0]:
print(bob._objects)
print(ada._objects)


{89204741720: tensor([2., 3., 5., 7.])}
{92501915015: tensor([2., 3., 5., 7.])}


In [0]:
tensor_pointer

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:74152989779 -> bob:89204741720]
	-> (Wrapper)>[PointerTensor | me:54754223715 -> ada:92501915015]

In [0]:
mytensor = tensor_pointer.get()
mytensor

[tensor([2., 3., 5., 7.]), tensor([2., 3., 5., 7.])]

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:25020039264 -> bob:11224473078]

In [0]:
y

(Wrapper)>[PointerTensor | me:12745297805 -> bob:85628357797]

In [0]:
z = x + y

In [0]:
z

(Wrapper)>[PointerTensor | me:51154600785 -> bob:85167755392]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:48051955889 -> bob:15973959475]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:81269568268 -> bob:98900356761]

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
# try this project here!
myinput = th.tensor([[0.,0], [1,0], [0,1], [1,1]], requires_grad=True)
input_ptr = myinput.send(ada)
target = th.tensor([[0.], [1], [0], [1]], requires_grad=True).send(ada)
weights = th.tensor([[0.], [0.]], requires_grad=True).send(ada)

In [0]:
for i in range(10):
    prediction = input_ptr.mm(weights)
    loss = ((prediction - target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad * 0.15)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.2600)
tensor(0.1202)
tensor(0.0588)
tensor(0.0288)
tensor(0.0141)
tensor(0.0069)
tensor(0.0034)
tensor(0.0017)
tensor(0.0008)


In [0]:
ada._objects

{2065750003: tensor([[0., 0.],
         [1., 0.],
         [0., 1.],
         [1., 1.]], requires_grad=True), 18576454044: tensor([[0.],
         [1.],
         [0.],
         [1.]], requires_grad=True), 64632393582: tensor([[0.9859],
         [0.0141]], requires_grad=True), 9083952076: tensor([[0.0000],
         [0.9798],
         [0.0202],
         [1.0000]], grad_fn=<MmBackward>)}

In [0]:

del input_ptr
ada.clear_objects()
ada._objects

{}

# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{26633497748: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{48657671214: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:24319539779 -> bob:85407615607]

In [0]:
bob._objects

{85407615607: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{85407615607: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{85407615607: tensor([1, 2, 3, 4, 5])}

In [0]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{80800486678: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
z = x + y
z

(Wrapper)>[PointerTensor | me:11061726075 -> bob:10585041333]

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

bob = sy.VirtualWorker(hook, 'bob')
ada = sy.VirtualWorker(hook, 'ada')

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(2.9897)
tensor(0.3496)
tensor(0.1136)
tensor(0.0702)
tensor(0.0492)
tensor(0.0353)
tensor(0.0255)
tensor(0.0185)
tensor(0.0135)
tensor(0.0100)
tensor(0.0074)
tensor(0.0055)
tensor(0.0041)
tensor(0.0031)
tensor(0.0023)
tensor(0.0017)
tensor(0.0013)
tensor(0.0010)
tensor(0.0007)
tensor(0.0006)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_ada = data[2:4].send(ada)
target_ada = target[2:4].send(ada)

In [0]:
datasets = [(data_bob, target_bob), (data_ada, target_ada)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [0]:
train()

tensor(0.2666, requires_grad=True)
tensor(1.8871, requires_grad=True)
tensor(0.6368, requires_grad=True)
tensor(1.0457, requires_grad=True)
tensor(0.4006, requires_grad=True)
tensor(0.6171, requires_grad=True)
tensor(0.2343, requires_grad=True)
tensor(0.3672, requires_grad=True)
tensor(0.1367, requires_grad=True)
tensor(0.2196, requires_grad=True)
tensor(0.0801, requires_grad=True)
tensor(0.1320, requires_grad=True)
tensor(0.0471, requires_grad=True)
tensor(0.0799, requires_grad=True)
tensor(0.0279, requires_grad=True)
tensor(0.0487, requires_grad=True)
tensor(0.0166, requires_grad=True)
tensor(0.0299, requires_grad=True)
tensor(0.0100, requires_grad=True)
tensor(0.0185, requires_grad=True)
tensor(0.0061, requires_grad=True)
tensor(0.0116, requires_grad=True)
tensor(0.0038, requires_grad=True)
tensor(0.0073, requires_grad=True)
tensor(0.0024, requires_grad=True)
tensor(0.0047, requires_grad=True)
tensor(0.0015, requires_grad=True)
tensor(0.0031, requires_grad=True)
tensor(0.0010, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [0]:
bob.clear_objects()
ada.clear_objects()

<VirtualWorker id:ada #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(ada)

In [0]:
bob._objects

{64606938033: tensor([1, 2, 3, 4, 5])}

In [0]:
ada._objects

{19845798845: (Wrapper)>[PointerTensor | ada:19845798845 -> bob:64606938033]}

In [0]:
y = x + x

In [0]:
y

(Wrapper)>[PointerTensor | me:59618635577 -> ada:20454240825]

In [0]:
bob._objects

{64606938033: tensor([1, 2, 3, 4, 5]),
 85286524610: tensor([ 2,  4,  6,  8, 10])}

In [0]:
ada._objects

{19845798845: (Wrapper)>[PointerTensor | ada:19845798845 -> bob:64606938033],
 20454240825: (Wrapper)>[PointerTensor | ada:20454240825 -> bob:85286524610]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
ada.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(ada)

In [0]:
bob._objects

{66644687680: tensor([1, 2, 3, 4, 5])}

In [0]:
ada._objects

{15493716599: (Wrapper)>[PointerTensor | ada:15493716599 -> bob:66644687680]}

In [0]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:15493716599 -> bob:66644687680]

In [0]:
bob._objects

{66644687680: tensor([1, 2, 3, 4, 5])}

In [0]:
ada._objects

{}

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

In [0]:
bob.clear_objects()
ada.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(ada)

In [0]:
bob._objects

{79028485993: tensor([1, 2, 3, 4, 5])}

In [0]:
ada._objects

{80735413772: (Wrapper)>[PointerTensor | ada:80735413772 -> bob:79028485993]}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
ada._objects

{}

# Lesson: Pointer Chain Operations

In [0]:
bob.clear_objects()
ada.clear_objects()

<VirtualWorker id:ada #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{8101064976: tensor([1, 2, 3, 4, 5])}

In [0]:
ada._objects

{}

In [0]:
x.move(ada)

(Wrapper)>[PointerTensor | me:28695579543 -> ada:28695579543]

In [0]:
bob._objects

{}

In [0]:
ada._objects

{28695579543: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(ada)

In [0]:
bob._objects

{9797992602: tensor([1, 2, 3, 4, 5])}

In [0]:
ada._objects

{28695579543: tensor([1, 2, 3, 4, 5]),
 62758017622: (Wrapper)>[PointerTensor | ada:62758017622 -> bob:9797992602]}

In [0]:
x.remote_get()

(Wrapper)>[PointerTensor | me:21008764540 -> ada:62758017622]

In [0]:
bob._objects

{}

In [0]:
ada._objects

{28695579543: tensor([1, 2, 3, 4, 5]), 62758017622: tensor([1, 2, 3, 4, 5])}

In [0]:
x.move(bob)

(Wrapper)>[PointerTensor | me:21008764540 -> bob:21008764540]

In [0]:
x

(Wrapper)>[PointerTensor | me:21008764540 -> bob:21008764540]

In [0]:
bob._objects

{21008764540: tensor([1, 2, 3, 4, 5])}

In [0]:
ada._objects

{28695579543: tensor([1, 2, 3, 4, 5])}